In [3]:
import numpy as np
from scipy.signal import correlate2d
import pygame
pygame.init()

GRIDWIDTH = 51      # количество клеток по горизонтали
GRIDHEIGHT = 51     # количество клеток по вертикали

clock = pygame.time.Clock()

CELLSIZE = 4       # размер клетки в пикселях
WINDOWWIDTH = GRIDWIDTH * CELLSIZE      # ширина окна
WINDOWHEIGHT = GRIDHEIGHT * CELLSIZE    # высота окна
FPS = 20
BGCOLOR = (35, 35, 35)

def drawRectangle(indices, A, B=None):
    valueA = np.clip(int(A * 255), 0, 255)
    # ограничили возможные значения valueA сверху и снизц
    if B is None:
        valueB = G = valueA
    else:
        valueB = np.clip(int(B * 255), 0, 255)
        G = 30
    
    color = (valueA, G, valueB)
    coords = (indices[0]*CELLSIZE, indices[1]*CELLSIZE)
    rect = (coords[0], coords[1], CELLSIZE, CELLSIZE)
    pygame.draw.rect(sf, color, rect)

# make a directory for figures:
!mkdir -p figures   

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file figures already exists.
Error occurred while processing: figures.


In [2]:
def underride(d, **options):
    """ добавляет key-value пары в словарь d, если key нет в d """
    for key, val in options.items():
        d.setdefault(key, val)



In [6]:
def f(**d):
    print(type(d))
f()

<class 'dict'>


#### Diffusion

In [2]:
array = np.random.rand(GRIDWIDTH, GRIDHEIGHT)
array = np.zeros((GRIDWIDTH, GRIDHEIGHT), dtype=float)

for i in range(GRIDWIDTH//2-1, GRIDWIDTH//2+2):
    for j in range(GRIDHEIGHT//2-1, GRIDHEIGHT//2+2):
        array[i, j] = 1   # центральная клетка будет 1

pygame.display.set_caption('Diffusion')

kernel = np.array([[0, 1, 0],
                   [1,-4, 1],
                   [0, 1, 0]])

c = correlate2d(array, kernel, mode='same', boundary='wrap')

sf = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))

iterCount = 10000
iterations = 0
running = True
while running:

    sf.fill(BGCOLOR)
    for i in range(GRIDWIDTH):
        for j in range(GRIDHEIGHT):
            drawRectangle((i, j), array[i, j], 0.5)

    c = correlate2d(array, kernel, mode='same', boundary='wrap')

    if iterations <= iterCount:
        array += 0.1 * c
        iterations += 1
    pygame.display.update()
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            running = False

#### Reaction-diffusion
For the RD model, we have two arrays, one for each chemical.

Following [Sims](http://www.karlsims.com/rd.html), I'm using a kernel that includes the diagonal elements.  They have lower weights because they are farther from the center cell.

The `step` function computes these functions:

$\Delta A = r_a \nabla^2 A - AB^2 + f (1-A) $

$\Delta B = r_b \nabla^2 B + AB^2 - (k+f) B $

where $\nabla^2$ is the Laplace operator the kernel is intended to approximate.

In [4]:
A = np.ones((GRIDWIDTH, GRIDHEIGHT), dtype=float)      # концентрация A
B = np.zeros((GRIDWIDTH, GRIDHEIGHT), dtype=float)     # концентрация B
rateA = 0.5     # скорость диффузии вещества A
rateB = 0.25    # скорость диффузии вещества B
f = 0.035         # скорость добавления вещества A в систему
k = 0.062         # скорость удаления вещества B из системы

value = 0.5     # значение концентрации B на островках
noise = 0.16     # уровень шума

def makeIsland(a, val, noise):
    n, m = a.shape
    r = min(n, m) // 20
    # в центре создается островок радиуса r с концентрацией val вещества:
    a[n//2-r : n//2+r, m//2-r : n//2+r] += val   
    a += noise * np.random.random((n, m))       # вносится шум в матрицу a

makeIsland(B, value, noise)

kernel = np.array([[.05, .2, .05],
                    [ .2, -1, .2],
                    [.05, .2, .05]])

sf = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))

running = True
while running:

    sf.fill(BGCOLOR)
    for i in range(GRIDWIDTH):
        for j in range(GRIDHEIGHT):
            drawRectangle((i, j), A[i, j], B[i, j])

    cA = correlate2d(A, kernel, mode='same', boundary='wrap')
    cB = correlate2d(B, kernel, mode='same', boundary='wrap')

    reaction = A * B**2

    A += rateA*cA - reaction + f*(1-A)
    B += rateB*cB + reaction - (f+k)*B

    # if iterations <= iterCount:
    #     array += 0.1 * c
    #     iterations += 1

        
    pygame.display.update()
    clock.tick(FPS)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            running = False

In [3]:
def drawRectangle(indices, A, B=None):
    valueA = np.clip(int(A * 255), 0, 255)
    if B is None:
        valueB = G = valueA
    else:
        valueB = np.clip(int(B * 255), 0, 255)
        G = 30
    
    color = (valueA, G, valueB)
    coords = (indices[0]*CELLSIZE, indices[1]*CELLSIZE)
    rect = (coords[0], coords[1], CELLSIZE, CELLSIZE)
    pygame.draw.rect(sf, color, rect)


6

In [9]:
a = 256898686
np.clip(a, 0, 255)

255

In [10]:
a = np.ones(5)
3-a

array([2., 2., 2., 2., 2.])